In [1]:
import pandas as pd
import os

workdir = "/Users/ns/cpet-ai/CPET-AI"
df_bxb_nevin = pd.read_parquet(os.path.join(workdir, "nevin_bxb_cleaned.parquet"))
df_bxb_sn = pd.read_parquet(os.path.join(workdir, "sean_yuewei_cleaned_cpet_df.parquet"))

In [2]:
# Compare columns between dataframes
print("Columns in df_bxb_nevin but not in df_bxb_sn:")
print(set(df_bxb_nevin.columns) - set(df_bxb_sn.columns))
print("\nColumns in df_bxb_sn but not in df_bxb_nevin:")
print(set(df_bxb_sn.columns) - set(df_bxb_nevin.columns))

print("\nData types comparison for common columns:")
common_cols = set(df_bxb_nevin.columns) & set(df_bxb_sn.columns)
comparison = pd.DataFrame({
    'df_bxb_nevin': df_bxb_nevin[common_cols].dtypes.astype(str),  # Convert dtypes to string
    'df_bxb_sn': df_bxb_sn[common_cols].dtypes.astype(str)         # Convert dtypes to string
})
print(comparison[comparison['df_bxb_nevin'] != comparison['df_bxb_sn']])

Columns in df_bxb_nevin but not in df_bxb_sn:
{'index', 'time_of_test', 'testDate', 'level_0', 'testTime', 'MRN'}

Columns in df_bxb_sn but not in df_bxb_nevin:
{'testDateTime'}

Data types comparison for common columns:
                 df_bxb_nevin                   df_bxb_sn
ExerciseDuration       object                     float64
Height                 object                     float64
AmbientRH              object                     float64
BarometricPress        object                     float64
DOB                    object  datetime64[ns, US/Pacific]
AmbientTemp            object                     float64
Weight                 object                     float64
TestDuration           object                     float64


In [21]:
# Localize time_of_test to Los Angeles timezone
df_bxb_nevin['time_of_test'] = pd.to_datetime(df_bxb_nevin['time_of_test']).dt.tz_localize('US/Pacific')
df_bxb_nevin.rename(columns={'time_of_test': 'testDateTime'}, inplace=True)


In [3]:
df_combined = pd.read_parquet(os.path.join(workdir, "combined_cleaned_df.parquet"))

In [18]:
time_last_char

0        930
1        868
2        918
3       1094
4       1089
        ... 
3877     914
3878     782
3879    1062
3880     620
3881    1056
Name: Time, Length: 3882, dtype: object

In [22]:
# Extract last character from Time column and compare with testDuration with tolerance
time_last_char = df_combined.Time.apply(lambda x: int(x[-1]))
mismatch_mask = abs(time_last_char - df_combined.TestDuration) > 20
print("Rows where Time last character doesn't match TestDuration (with tolerance of 5):")
print(df_combined[mismatch_mask][['Time', 'TestDuration']])
print(f"\nTotal mismatches: {mismatch_mask.sum()}")

Rows where Time last character doesn't match TestDuration (with tolerance of 5):
                                                   Time  TestDuration
213   [0, 3, 8, 12, 15, 26, 29, 40, 49, 56, 62, 65, ...        1020.0
214   [1, 3, 6, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25...         647.0
960   [3, 5, 8, 12, 16, 20, 22, 26, 29, 32, 35, 38, ...         940.0
1095                      [0, 3, 8, 12, 15, 18, 22, 25]         860.0
1334  [1, 8, 16, 22, 26, 30, 34, 37, 41, 46, 52, 58,...        2580.0
2021  [1, 5, 9, 13, 16, 21, 26, 35, 42, 50, 57, 60, ...        1136.0
2380  [3, 7, 11, 15, 20, 24, 27, 31, 35, 39, 45, 48,...        1020.0
3656  [2, 6, 8, 12, 13, 17, 20, 23, 26, 29, 33, 36, ...           0.0

Total mismatches: 8


In [27]:
time_last_char.iloc[214]

680